![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

# Concordancing with R

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Concordancing with R*](https://ladal.edu.au/kwics.html). 


***


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial.


In [ ]:
# activate packages
library(quanteda) # for concordancing
library(dplyr)    # for table processing
library(stringr)  # for text processing
library(writexl)  # for saving data
library(here)     # for easy pathing


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("quanteda")`. Then, you simply run this command and R will install the package you specified.


## Loading and processing textual data

For this tutorial, the default data represents the text of Lewis Caroll's  *Alice's Adventures in Wonderland* which we download from the [GitHub data repository of the *Language Technology and Data Analysis Laboratory* (LADAL)](https://slcladal.github.io/data). 

***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then, when the menu has unfolded, click on the smaller folder symbol (encircled in red in the picture below).

![Small Binder Folder Symbol](https://slcladal.github.io/images/upload2.png)

Now, you are in the main menu and can click on the 'MyData' folder.

![MyData Folder Symbol](https://slcladal.github.io/images/upload3.png)

Now, that you are in the MyData folder, you can click on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Select and upload the files you want to analyze (**IMPORTANT: here, we assume that you upload some form of text data - not tabular data! You can upload only txt and docx files!**). When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.


In [ ]:
myfiles <- list.files(here::here("MyData"), # path to the corpus data
                          # full paths - not just the names of the files
                          full.names = T) 
# load files
mytext <- sapply(myfiles, function(x){
  x <- scan(x, 
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T)
  x <- paste0(x, sep = " ", collapse = " ")
  x <- stringr::str_squish(x)
})
# inspect
str(mytext)


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***

If you do not use your own data, you can continue with the default data, Lewis Caroll's  *Alice's Adventures in Wonderland*, which we load by running the code below (but you have to have access to the internet to do so).


In [ ]:
text <- base::readRDS(url("https://slcladal.github.io/data/alice.rda", "rb"))
# inspect
head(text)


The inspection of the data shows that the data consists of a vector of individual strings which contain the example text. This means that the data requires formatting so that we can use it. Therefore, we collapse it into a single object (or text),  remove superfluous white spaces, and then tokenize the data (tokenizing means that we split it into individual  tokens or words.



In [ ]:
text <- text %>%
  # collapse lines into a single  text
  paste0(collapse = " ") %>%
  # remove superfluous white spaces
  str_squish() %>%
  # tokenize
  tokens()
# inspect
head(text)


The result confirms that the entire text is now split into individual words. 

## Creating simple concordances

Now that we have loaded the data, we can easily extract concordances using the `kwic` function from the `quanteda` package. The `kwic` function takes the text (`x`) and the search pattern (`pattern`) as it main arguments but it also allows the specification of the context window, i.e. how many words/elements are show to the left and right of the key word (we will go over this later on).


In [ ]:
mykwic <- kwic(
  # define text
  text, 
  # define target word (this is called the "search pattern")
  pattern = "alice")
# inspect
mykwic %>%
  as.data.frame() %>%
  head()


We can easily extract the frequency of the search term (*alice*) using the `nrow` or the `length` functions which provide the number of rows of a tables (`nrow`) or the length of a vector (`length`).



In [ ]:
nrow(mykwic); length(mykwic$keyword)



The results show that there are 386 instances of the search term (*alice*). To get a better understanding of the use of a word, it is often useful to extract more context. This is easily done by increasing size of the context window. To do this, we specify the `window` argument of the `kwic` function. In the example below, we set the context window size to 10 words/elements rather than using the default (which is 5 word/elements).



In [ ]:
mykwic_longer <- kwic(text, pattern = "alice", 
  # define context window size
  window = 10)
# inspect
mykwic_longer %>%
  as.data.frame() %>%
  head()


## Exporting concordances

To export a concordance table as an MS Excel spreadsheet, you can use the `write_xlsx` function from the `writexl` package as shown below. Be aware that we use the `here` function from the `here` package to define where we want to save the file (in this case this will be in the current working directory).


In [ ]:
write_xlsx(mykwic, here::here("mykwic.xlsx"))



## Extracting more than single words

While extracting single words is very common, you may want to extract more than just one word. To extract phrases, all you need to so is to specify that the pattern you are looking for is a phrase, as shown below.


In [ ]:
kwic_pooralice <- kwic(text, pattern = phrase("poor alice"))
# inspect
kwic_pooralice %>%
  as.data.frame() %>%
  head()


Of course you can extend this to longer sequences such as entire sentences. However, you may want to extract more or less concrete patterns rather than words or phrases. To search for patterns rather than words, you need to include regular expressions in your search pattern. 



## Searches using regular expressions

Regular expressions allow you to search for abstract patterns rather than concrete words or phrases which provides you with an extreme flexibility in what you can retrieve. A regular expression (in short also called *regex* or *regexp*) is a special sequence of characters that stand for are that describe a pattern. For more information about regular expression in R [see this tutorial](https://ladal.edu.au/regex.html).


To include regular expressions in your KWIC searches, you include them in your search pattern and set the argument `valuetype` to `"regex"`. The search pattern `"\\balic.*|\\bhatt.*"` retrieves elements that contain `alic` and `hatt` followed by any characters and where the `a` in `alic` and the `h` in `hatt` are at a word boundary, i.e. where they are the first letters of a word. Hence, our search would not retrieve words like *malice* or *shatter*. The `|` is an operator (like `+`, `-`, or `*`) that stands for *or*.


In [ ]:
# define search patterns
patterns <- c("\\balic.*|\\bhatt.*")
kwic_regex <- kwic(text, patterns, 
  # define valuetype
  valuetype = "regex")
# inspect
kwic_regex %>%
  as.data.frame() %>%
  head()


## Piping concordances

Quite often, we only want to retrieve patterns if they occur in a certain context. For instance, we might be interested in instances of *alice* but only if the preceding word is *poor*. Such conditional concordances could be extracted using regular expressions but they are easier to retrieve by piping. Piping is done using the `%>%` function from the `dplyr` package and the piping sequence can be translated as *and then*. We can then filter those concordances that contain *poor* using the `filter` function from the `dplyr` package. Note the the `$` stands for the end of a string so that *poor$* means that *poor* is the last element in the string that is preceding the keyword.


In [ ]:
kwic_pipe <- kwic(x = text, pattern = "alice") %>%
  dplyr::filter(stringr::str_detect(pre, "poor$|little$"))
# inspect
kwic_pipe %>%
  as.data.frame() %>%
  head()


Piping is a very useful helper function and it is very frequently used in R - not only in the context of text processing but in all data science related domains.

## Arranging concordances and adding frequency information

When inspecting concordances, it is useful to re-order the concordances so that they do not appear in the order that they appeared in the text or texts but by the context. To reorder concordances, we can use the `arrange` function from the `dplyr` package which takes the column according to which we want to re-arrange the data as it main argument. 

In the example below, we extract all instances of *alice* and then arrange the instances according to the content of the `post` column in alphabetical.


In [ ]:
kwic_ordered <- kwic(x = text, pattern = "alice") %>%
  dplyr::arrange(post)
# inspect
kwic_ordered %>%
  as.data.frame() %>%
  head() 


Arranging concordances according to alphabetical properties may, however, not be the most useful option. A more useful option may be to arrange concordances according to the frequency of co-occurring terms or collocates. In order to do this, we need to extract the co-occurring words and calculate their frequency. We can do this by combining the  `mutate`, `group_by`, `n()` functions from the `dplyr` package with the `str_remove_all` function from the `stringr` package. Then, we arrange the concordances by the frequency of the collocates in descending order (that is why we put a `-` in the arrange function). In order to do this, we need to 

1. create a new variable or column which represents the word that co-occurs with, or, as in the example below, immediately follows the search term. In the example below, we use the `mutate` function to create a new column called `post_word`. We then use the `str_remove_all` function to remove everything except for the word that immediately follows the search term (we simply remove everything and including a white space).

2. group the data by the word that immediately follows the search term.

3. create a new column called `post_word_freq` which represents the frequencies of all the words that immediately follow the search term.

4. arrange the concordances by the frequency of the collocates in descending order.


In [ ]:
kwic_ordered_coll <- kwic(
  # define text
  x = text, 
  # define search pattern
  pattern = "alice") %>%
  # extract word following the keyword
  dplyr::mutate(post_word = str_remove_all(post, " .*")) %>%
  # group following words
  dplyr::group_by(post_word) %>%
  # extract frequencies of the following words
  dplyr::mutate(post_word_freq = n()) %>%
  # arrange/order by the frequency of the following word
  dplyr::arrange(-post_word_freq)
# inspect
kwic_ordered_coll %>%
  as.data.frame() %>%
  head()


We add more columns according to which we could arrange the concordance following the same schema. For example, we could add another column that represented the frequency of words that immediately preceded the search term and then arrange according to this column.

## Ordering by subsequent elements

In this section, we will extract the three words following the keyword (*alice*) and organize the concordances by the frequencies of the following words. We begin by inspecting the first 6 lines of the concordance of *alice*.


In [ ]:
head(mykwic)



Next, we take the concordances and create a clean post column that is all in lower case and that does not contain any punctuation.



In [ ]:
mykwic %>%
  # convert to data frame
  as.data.frame() %>%
  # create new CleanPost
  dplyr::mutate(CleanPost = stringr::str_remove_all(post, "[:punct:]"),
                CleanPost = stringr::str_squish(CleanPost),
                CleanPost = tolower(CleanPost))-> kwic_alice_following
# inspect
head(mykwic_following)


In a next step, we extract the 1^st^, 2^nd^, and 3^rd^ words following the keyword.



In [ ]:
mykwic_following %>%
  # extract first element after keyword
  dplyr::mutate(FirstWord = stringr::str_remove_all(CleanPost, " .*")) %>%
  # extract second element after keyword
  dplyr::mutate(SecWord = stringr::str_remove(CleanPost, ".*? "),
                SecWord = stringr::str_remove_all(SecWord, " .*")) %>%
  # extract third element after keyword
  dplyr::mutate(ThirdWord = stringr::str_remove(CleanPost, ".*? "),
                ThirdWord = stringr::str_remove(ThirdWord, ".*? "),
                ThirdWord = stringr::str_remove_all(ThirdWord, " .*")) -> kwic_alice_following
# inspect
head(mykwic_following)


Next, we calculate the frequencies of the subsequent words and order in descending order from the  1^st^ to the 3^rd^ word following the keyword.



In [ ]:
mykwic_following %>%
  # calculate frequency of following words
  # 1st word
  dplyr::group_by(FirstWord) %>%
  dplyr::mutate(FreqW1 = n()) %>%
  # 2nd word
  dplyr::group_by(SecWord) %>%
  dplyr::mutate(FreqW2 = n()) %>%
  # 3rd word
  dplyr::group_by(ThirdWord) %>%
  dplyr::mutate(FreqW3 = n()) %>%
  # ungroup
  dplyr::ungroup() %>%
  # arrange by following words
  dplyr::arrange(-FreqW1, -FreqW2, -FreqW3) -> mykwic_following
# inspect results
head(mykwic_following, 10)


The results now show the concordance arranged by the frequency of the words following the keyword.

***

[Back to LADAL](https://ladal.edu.au/kwics.html)

***
